In [16]:
library(tidyverse)
library(lubridate)
library(forecast) # Para modelos AR

In [192]:
df<-read_csv('df.csv',col_names = TRUE,cols(
  Ano = col_double(),
  Mes = col_double(),
  Ano_Mes = col_character(),
  Fecha = col_date(format = ""),
  Precios_Mensuales = col_double(),
  Aportes_Energia_GWh = col_double(),
  Volumen_Util_Diario_Energia_GWh = col_double(),
  Demanda_Energia_SIN_GWh = col_double(),
  ONI = col_double(),
  CEN_Total = col_double(),
  CEN_Hidro = col_double(),
  CEN_Termica = col_double()
))

# Creacion de funciones

In [179]:
oni<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    
    n_pronosticos<-df%>%filter(Fecha>=start_at)%>%select(ONI)%>%nrow
    
    
    while(any(is.na(df$ONI))){
        fila<-sum(complete.cases(df$ONI))+1
        m<-auto.arima(unlist(unname(df[1:fila-1,'ONI'])),max.q = 0,max.Q = 0,max.d = 0,max.D = 0)
        y_est<-round(as.vector(unlist(forecast(m, 24)['mean'])),1)
        df$ONI[is.na(df$ONI)]<-y_est
    }
    #df$ONI[df$Fecha>=as.Date(start_at)]<-round(runif(n_pronosticos,-0.5,0.5),1)
    
    return(df)
}
#ok

In [180]:
aportes<-function(df, start_at, rezagos_oni=12,rezagos_aportes=24){
    
    fechaini<-as.Date(start_at)
    #n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Aportes_Energia_GWh)%>%nrow
    #df$Aportes_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Aportes_Energia_GWh,na.rm = TRUE),max(df$Aportes_Energia_GWh,na.rm = TRUE)),3)

    fecha_forecast<-seq.Date(from = as.Date(fechaini),length.out = 24,by = 'month')  # Meses para iterar

    for(i in seq(length(fecha_forecast))){                                       # Inicia el pronostico un paso adelante
    
        temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados) 
        rezagos_oni<-rezagos_oni                                                #Rezagos para el ONI (parametrizable)
        rezagos_aportes<-rezagos_aportes                                        #Rezagos para los Aportes (parametrizable)

        for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
            col_oni<-paste0('ONI_',j)                                           #   
            temp[[col_oni]]<-lag(df$ONI,j)                                      #
        }                                                                       #

        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #

        new_data<-temp
        
        
        
        temp<-temp%>%select(-1,-2,-3,Aportes_Normalizados,                      #Elimina Fecha, Aportes_Energia, ONI
                                    starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
                                    starts_with('ONI_'))                        #Trae los rezagos de ONI


        temp<-temp%>%drop_na()                                                  #Elimina las fechas con valores NA

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Aportes_Normalizados~.,data = temp)                        #Ajusta un modelo con todas las vars
        m<-(step(lm(Aportes_Normalizados~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01                                 #Define el parametro de tolerancia
        relevant.x <- names(toselect.x)[toselect.x == TRUE]                  
        sig.formula <- as.formula(paste("Aportes_Normalizados ~",paste(relevant.x,collapse = '+'))) 

        m<-lm(formula = sig.formula,data = temp)                                #Ajusta el modelo final
        
        y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[i]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
        
        df$Aportes_Normalizados[df$Fecha==fecha_forecast[i]]<-y_est

    }
    
    return(df)
}

In [181]:
reservas<-function(df, start_at, rezagos_aportes, rezagos_oni,rezagos_reserva, rezagos_termica_hidrica){
    
    fechaini<-as.Date(start_at)
    #n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Volumen_Util_Diario_Energia_GWh)%>%nrow
    #df$Volumen_Util_Diario_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),max(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),3)
    
    fecha_forecast<-seq.Date(from = as.Date(fechaini),length.out = 24,by = 'month')  # Meses para iterar

    for(i in seq(length(fecha_forecast))){                                       # Inicia el pronostico un paso adelante
    
        temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados,Reservas_Normalizadas) 
        
        rezagos_oni<-rezagos_oni                                                #Rezagos para el ONI (parametrizable)
        rezagos_aportes<-rezagos_aportes                                        #Rezagos para los Aportes (parametrizable)
        rezagos_reserva<-rezagos_reserva
        rezagos_termica_hidrica<-rezagos_termica_hidrica
        
        for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
            col_oni<-paste0('ONI_',j)                                           #   
            temp[[col_oni]]<-lag(df$ONI,j)                                      #
        }                                                                       #

        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #
        
        for(p in seq(rezagos_reserva)){                                         #Creacion de columnas rezagadas Aportes
            col_reservas<-paste0('Reservas_',p)                                 #
            temp[[col_reservas]]<-lag(df$Reservas_Normalizadas,p)               #
        }                                                                       #
        
        for(p in seq(rezagos_termica_hidrica)){                                         #Creacion de columnas rezagadas Aportes
            col_calc<-paste0('Term_Hidraulica_',p)                                 #
            temp[[col_calc]]<-lag(df$Termica_Hidraulica,p)               #
        }                                                                       #



        new_data<-temp
        
        
        
        temp<-temp%>%select(-1,-2,-3,
                                    -Aportes_Normalizados,
                                    Reservas_Normalizadas,                     #Elimina Fecha, Aportes_Energia, ONI
                                    starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
                                    starts_with('Reservas_[0-9]'),              #Trae los rezagos de Reservas
                                    starts_with('Term_Hidraulica_[0-9]'),       #Trae los rezagos de Term_Hidraulica (calculada)
                                    starts_with('ONI_'))                        #Trae los rezagos de ONI


        temp<-temp%>%drop_na()                                                  #Elimina las fechas con valores NA

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Reservas_Normalizadas~.,data = temp)                        #Ajusta un modelo con todas las vars
        m<-(step(lm(Reservas_Normalizadas~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01                                 #Define el parametro de tolerancia
        relevant.x <- names(toselect.x)[toselect.x == TRUE]                  
        sig.formula <- as.formula(paste("Reservas_Normalizadas ~",paste(relevant.x,collapse = '+'))) 

        m<-lm(formula = sig.formula,data = temp)                                #Ajusta el modelo final
        y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[i]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
        
        df$Reservas_Normalizadas[df$Fecha==fecha_forecast[i]]<-y_est

    }


    
    return(df)
}

In [182]:
df%>%tail(1)#%>%select(Fecha, ONI,Aportes_Normalizados,Reservas_Normalizadas)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica
2021,5,2021-05,2021-05-01,NA,NA,NA,NA,NA,13.16023,8.68223,4.464


In [183]:
fecha_forecast<-seq.Date(from = as.Date('2016-06-01'),length.out = 24,by = 'month')  # Meses para iterar

        

        temp<-df%>%select(Fecha,  Aportes_Normalizados,Reservas_Normalizadas, Demanda_Energia_SIN_GWh, Dem_CEN, Precios_Mensuales) 
        
        rezagos_precios<-24
        rezagos_aportes<-24                                        #Rezagos para los Aportes (parametrizable)
        rezagos_reserva<-24
        rezagos_demanda<-24
        rezagos_dem_cen<-24

        
         for(k in seq(rezagos_precios)){                                         #Creacion de columnas rezagadas Aportes
            col_precios<-paste0('Precios_',k)                                   #
            temp[[col_precios]]<-lag(df$Precios_Mensuales,k)                 #
        }                                                                       #


        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #
        
        for(p in seq(rezagos_reserva)){                                         #Creacion de columnas rezagadas Aportes
            col_reservas<-paste0('Reservas_',p)                                 #
            temp[[col_reservas]]<-lag(df$Reservas_Normalizadas,p)               #
        }                                                                       #
        
        for(p in seq(rezagos_demanda)){                                         #Creacion de columnas rezagadas Aportes
            col_demanda<-paste0('Demanda_GWh_',p)                                  #
            temp[[col_demanda]]<-lag(df$Demanda_Energia_SIN_GWh,p)                 #
        }                                                                       #


        for(p in seq(rezagos_dem_cen)){                                         #Creacion de columnas rezagadas Aportes
            col_dem_cen<-paste0('Dem_cen_',p)                                      #
            temp[[col_dem_cen]]<-lag(df$Dem_CEN,p)                                 #
        }                                                                       #



        new_data<-temp

        

        temp<-temp%>%select(-1,-2,-3,-4,-5, Precios_Mensuales, 
                                      starts_with('Precios_'),
                                      starts_with('Aportes_[0-9]'),
                                      starts_with('Reservas_[0-9]'),
                                      starts_with('Demanda_GWh_[0-9]'),
                                      starts_with('Dem_cen_[0-9]'))
        temp<-temp%>%drop_na()

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Precios_Mensuales~.,data = temp)                        #Ajusta un modelo con todas las vars
        m<-(step(lm(Precios_Mensuales~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01                                 #Define el parametro de tolerancia
        relevant.x <- names(toselect.x)[toselect.x == TRUE]
        relevant.x<-relevant.x[relevant.x != '(Intercept)']
        sig.formula <- as.formula(paste("Precios_Mensuales ~",paste(relevant.x,collapse = '+'))) 

         m<-lm(formula = sig.formula,data = temp)                                #Ajusta el modelo final
         y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[1]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
         df$Precios_Mensuales[df$Fecha==fecha_forecast[1]]<-y_est


ERROR: Error in .f(.x[[i]], ...): object 'Aportes_Normalizados' not found


In [194]:
precios<-function(df, start_at,
                  rezagos_precios,
                  rezagos_aportes,
                  rezagos_reserva,
                  rezagos_demanda,
                  rezagos_dem_cen){
    
    fechaini<-as.Date(start_at)
    #n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Precios_Mensuales)%>%nrow
    #df$Precios_Mensuales[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Precios_Mensuales,na.rm = TRUE),max(df$Precios_Mensuales,na.rm = TRUE)),3)
    
    fecha_forecast<-seq.Date(from = as.Date(fechaini),length.out = 24,by = 'month')  # Meses para iterar

    for(i in seq(length(fecha_forecast))){                                       # Inicia el pronostico un paso adelante

        temp<-df%>%select(Fecha,  Aportes_Normalizados,Reservas_Normalizadas, Demanda_Energia_SIN_GWh, Dem_CEN, Precios_Mensuales) 
        
        rezagos_precios<-24
        rezagos_aportes<-24                                        #Rezagos para los Aportes (parametrizable)
        rezagos_reserva<-24
        rezagos_demanda<-24
        rezagos_dem_cen<-24

        
         for(k in seq(rezagos_precios)){                                         #Creacion de columnas rezagadas Aportes
            col_precios<-paste0('Precios_',k)                                   #
            temp[[col_precios]]<-lag(df$Precios_Mensuales,k)                 #
        }                                                                       #


        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #
        
        for(p in seq(rezagos_reserva)){                                         #Creacion de columnas rezagadas Aportes
            col_reservas<-paste0('Reservas_',p)                                 #
            temp[[col_reservas]]<-lag(df$Reservas_Normalizadas,p)               #
        }                                                                       #
        
        for(p in seq(rezagos_demanda)){                                         #Creacion de columnas rezagadas Aportes
            col_demanda<-paste0('Demanda_GWh_',p)                                  #
            temp[[col_demanda]]<-lag(df$Demanda_Energia_SIN_GWh,p)                 #
        }                                                                       #


        for(p in seq(rezagos_dem_cen)){                                         #Creacion de columnas rezagadas Aportes
            col_dem_cen<-paste0('Dem_cen_',p)                                      #
            temp[[col_dem_cen]]<-lag(df$Dem_CEN,p)                                 #
        }                                                                       #



        new_data<-temp

        

        temp<-temp%>%select(-1,-2,-3,-4,-5, Precios_Mensuales, 
                                      starts_with('Precios_'),
                                      starts_with('Aportes_[0-9]'),
                                      starts_with('Reservas_[0-9]'),
                                      starts_with('Demanda_GWh_[0-9]'),
                                      starts_with('Dem_cen_[0-9]'))
        temp<-temp%>%drop_na()

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Precios_Mensuales~.,data = temp)                        #Ajusta un modelo con todas las vars
        m<-(step(lm(Precios_Mensuales~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01                                 #Define el parametro de tolerancia
        relevant.x <- names(toselect.x)[toselect.x == TRUE]
        relevant.x<-relevant.x[relevant.x != '(Intercept)']
        sig.formula <- as.formula(paste("Precios_Mensuales ~",paste(relevant.x,collapse = '+'))) 

         m<-lm(formula = sig.formula,data = temp)                                #Ajusta el modelo final
         y_est<-round(as.vector(unlist(unname(forecast(m,h = 1,newdata = new_data%>%filter(Fecha==fecha_forecast[i]%m+%months(-1))%>%select(Fecha, one_of(names(summary(m)$coeff[-1,1])))%>%select(-Fecha))['mean']))),4)
         df$Precios_Mensuales[df$Fecha==fecha_forecast[i]]<-y_est
        

    }


    
    return(df)
}

In [195]:
demanda<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Demanda_Energia_SIN_GWh)%>%nrow
    df$Demanda_Energia_SIN_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Demanda_Energia_SIN_GWh,na.rm = TRUE),max(df$Demanda_Energia_SIN_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [196]:
columnas_calculadas<-function(df){
    #debo montar dem_CEN
    df$Dem_CEN<-round(df$Demanda_Energia_SIN_GWh/(as.vector(days_in_month(df$Fecha))*df$CEN_Total*24),4)
    #debo montar Term_Hidro
    df$Termica_Hidraulica<-round(df$CEN_Termica/df$CEN_Hidro,4)
    
    return(df)
}

In [197]:
normalizar_hidrologia<-function(df){
    df<-df%>%
        left_join(
                df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh,na.rm = TRUE), 
                                                              mean_reservas = mean(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),
                                                              sd_aportes = sd(Aportes_Energia_GWh, na.rm = TRUE),
                                                              sd_reservas = sd(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),by='Mes')%>%
    mutate(
        Aportes_Normalizados = (Aportes_Energia_GWh-mean_aportes)/sd_aportes,
        Reservas_Normalizadas = (Volumen_Util_Diario_Energia_GWh-mean_reservas)/sd_reservas)
    
    return(df)
}

# Probar funciones

In [198]:
df<-normalizar_hidrologia(df)
df<-demanda(df,start_at = '2019-06-01')
df<-columnas_calculadas(df)
df<-oni(df,start_at = '2019-06-01')
df<-aportes(df,start_at = '2019-06-01',rezagos_oni = 12,rezagos_aportes = 24)
df<-reservas(df,start_at = '2019-06-01', rezagos_oni = 12,rezagos_aportes = 24,rezagos_reserva = 24,rezagos_termica_hidrica = 12)
df<-precios(df,start_at = '2019-06-01',rezagos_precios = 24,rezagos_aportes = 24,rezagos_reserva = 24,rezagos_demanda = 24,rezagos_dem_cen = 24)

ERROR: Error in mutate_impl(.data, dots): Evaluation error: object 'mean_aportes' not found.


In [174]:
df%>%tail(5)#%>%select(Fecha, ONI,Aportes_Normalizados,Reservas_Normalizadas)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica,mean_aportes,mean_reservas,sd_aportes,sd_reservas,Aportes_Normalizados,Reservas_Normalizadas,Dem_CEN,Termica_Hidraulica
2021,1,2021-01,2021-01-01,NA,NA,NA,5123.809,-0.1,13.15767,8.67967,4.464,2387.357,10418.906,860.5643,1018.8277,0.3756,-0.1829,NA,0.5143
2021,2,2021-02,2021-02-01,NA,NA,NA,5816.269,0.0,13.15867,8.68067,4.464,1977.469,8931.573,571.9035,947.0586,0.3597,-0.2316,NA,0.5142
2021,3,2021-03,2021-03-01,NA,NA,NA,3974.901,0.0,13.15867,8.68067,4.464,2921.435,7971.466,1105.0457,1191.9835,0.3392,-0.2086,NA,0.5142
2021,4,2021-04,2021-04-01,NA,NA,NA,5594.153,0.0,13.15867,8.68067,4.464,4456.870,8298.807,1832.5230,1671.8904,0.3010,-0.2603,NA,0.5142
2021,5,2021-05,2021-05-01,NA,NA,NA,5046.454,0.0,13.16023,8.68223,4.464,6027.007,9605.625,1735.7382,1865.8465,0.2774,-0.2338,NA,0.5142


In [201]:
generar_archivos<-function(df,
                           start_at,
                           rezagos_precios,
                           rezagos_oni,
                           rezagos_aportes,
                           rezagos_reserva,
                           rezagos_demanda,
                           rezagos_termica_hidrica,
                           rezagos_dem_cen,
                           n_iter,
                           folder_location){
    
    
    if(dir.exists(folder_location)){
        while(file.remove(list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1])==TRUE){
            list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1]
        }

        fechaini<-as.Date(start_at)

        for( i in seq(n_iter)){
            ini<-now()
#             #Normalizar aportes
#             #normalizar reservas
#             #Agregar variables calculadas
#             df<-precios(df,fechaini)
#             df<-oni(df,fechaini)
#             df<-aportes(df,fechaini)
#             df<-reservas(df,fechaini)
#             df<-demanda(df,fechaini)
#             df<-columnas_calculadas(df)
            df<-read_csv('df.csv',col_names = TRUE,cols(
                          Ano = col_double(),
                          Mes = col_double(),
                          Ano_Mes = col_character(),
                          Fecha = col_date(format = ""),
                          Precios_Mensuales = col_double(),
                          Aportes_Energia_GWh = col_double(),
                          Volumen_Util_Diario_Energia_GWh = col_double(),
                          Demanda_Energia_SIN_GWh = col_double(),
                          ONI = col_double(),
                          CEN_Total = col_double(),
                          CEN_Hidro = col_double(),
                          CEN_Termica = col_double()
                        ))
            
            df<-normalizar_hidrologia(df)
            df<-demanda(df,start_at = start_at)
            df<-columnas_calculadas(df)
            df<-oni(df,start_at = start_at)
            df<-aportes(df,start_at = start_at ,rezagos_oni = rezagos_oni,rezagos_aportes = rezagos_aportes)
            df<-reservas(df,
                         start_at = start_at,
                         rezagos_oni = rezagos_oni,
                         rezagos_aportes = rezagos_aportes,
                         rezagos_reserva = rezagos_reserva,
                         rezagos_termica_hidrica = rezagos_termica_hidrica)

            df<-precios(df,start_at = start_at,
                        rezagos_precios = rezagos_precios,
                        rezagos_aportes = rezagos_aportes,
                        rezagos_reserva = rezagos_reserva,
                        rezagos_demanda = rezagos_demanda,
                        rezagos_dem_cen = rezagos_dem_cen)

            #pronostico de variables calculadas
            write_csv(df, paste0(folder_location,str_pad(i, pad = 0, width = 5),'.csv'))
            fin<-now()
            cat(paste0(round(fin-ini,4), ' segundos ',str_pad(i, pad = 0, width = 5),'.csv'),collapse='\n')
        }
        cat(paste0(n_iter, ' archivos generados en la ruta ',folder_location))
    } else{
        cat('Error: Directorio no existe en esta máquina')
    }

}

In [203]:
generar_archivos(df,start_at = '2019-06-01', 
                                             rezagos_precios =24,
                                             rezagos_oni = 12, 
                                             rezagos_aportes = 24,
                                             rezagos_reserva = 24,
                                             rezagos_demanda = 24,
                                             rezagos_dem_cen = 24,
                                             rezagos_termica_hidrica=24,
                                             n_iter = 2,
                                             folder_location = './Resultados/')

42.1757 segundos 00001.csv 
45.5832 segundos 00002.csv 
2 archivos generados en la ruta ./Resultados/

# Tareas

In [ ]:
#hacer el arx pronosticar uno a uno y tiene como entrada el ONI y se devuelven las unidades 2h
#hacer arx para reservas con oni, aportes, relacion T/H y me devuelvo 2h
#un modelo simple de precio 2h